# Required imports

In [1]:
from langchain_community.llms.llamafile import Llamafile
from langchain.prompts import PromptTemplate
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter

In [2]:
# Initialize Llamafile
llm = Llamafile(temperature=0.7)

# Code Samples

## Using Default Prompts

In [3]:
prompt_template = PromptTemplate.from_template("What is GenAI?")

prompt = prompt_template.invoke({})
result = llm.invoke(prompt)
print(result)


2. How does GenAI work?
3. What is the potential of GenAI in the future?
4. Who are some examples of successful GenAI applications?
5. Can you provide some examples of how GenAI has improved productivity and efficiency in industries such as medicine, manufacturing, and finance?
6. How can investors or businesses get involved with GenAI projects?</s>


In [4]:
# Load local data
pdf_path = '../../../../../_001.Data/sample-pdf/sample-pdf-file.pdf'
pdf_loader = PyPDFLoader(file_path = pdf_path)

In [5]:
# Default load and split
# split is based on the pages
pdf_data = pdf_loader.load_and_split()
print(f"Number of pages in pdf: {len(pdf_data)}")
# Print content for one of the pages
print(f"\nPage Content: {pdf_data[0].page_content}")

Number of pages in pdf: 5

Page Content: At the Española Healing Foods Oasis in Española, New Mexico, Pueblo dryland farming techniques are on display in a downtown public park. The garden, designed and planted by the Indigenous-led organization Tewa Women United, demonstrates how food and medicine can be grown in an environment that receives just 11 inches of rain per year. And at a nearby community garden, which the organization helped operate in the past, Pueblo members and locals grow fruit and vegetables.The garden projects are part of the organization’s efforts to grow foods and herbs for people in the Eight Northern Indian Pueblos, as well as locals in the wider Española Valley, using traditional methods. But there’s a problem: The soils at these gardens are being exposed to contaminants. Tewa Women United hopes oyster mushrooms will clean them up.At the community garden, a 2015 study found levels of contaminants high enough to threaten human health. Similarly toxic levels of th

In [6]:
# Split using Recursive splitter
# split is based on the specified chunk size
splitter = RecursiveCharacterTextSplitter(chunk_size = 1000,chunk_overlap = 0)

pdf_data_RS = pdf_loader.load_and_split(text_splitter=splitter)
print(f"Number of chunk: {len(pdf_data_RS)}")
# Print content for one of the chunks
print(f"\nPage Content: {pdf_data_RS[0].page_content}")

Number of chunk: 9

Page Content: At the Española Healing Foods Oasis in Española, New Mexico, Pueblo dryland farming techniques are on display in a downtown public park. The garden, designed and planted by the Indigenous-led organization Tewa Women United, demonstrates how food and medicine can be grown in an environment that receives just 11 inches of rain per year. And at a nearby community garden, which the organization helped operate in the past, Pueblo members and locals grow fruit and vegetables.The garden projects are part of the organization’s efforts to grow foods and herbs for people in the Eight Northern Indian Pueblos, as well as locals in the wider Española Valley, using traditional methods. But there’s a problem: The soils at these gardens are being exposed to contaminants. Tewa Women United hopes oyster mushrooms will clean them up.At the community garden, a 2015 study found levels of contaminants high enough to threaten human health. Similarly toxic levels of these or 

## Using Custom Prompts

### [Completion Prompts](https://docs.llamaindex.ai/en/stable/examples/customization/prompts/completion_prompts/)

In [7]:
from langchain.chains.summarize import load_summarize_chain
# create prompt
template = """
Write a summary that highlights the main ideas in 3 bullet points of the following:
"{text}"
SUMMARY:
"""

prompt = PromptTemplate.from_template(template)

# Instantiate chain
chain = load_summarize_chain(
    llm=llm,
    chain_type='stuff',
    prompt=prompt,
    verbose=False   # Setting this to true will print the formatted prompt
)

In [8]:
# Invoke chain
results = chain.invoke(pdf_data_RS)
print(f"Result Keys: {results.keys()}")
print(f"\nOutput: {results['output_text']}")

Result Keys: dict_keys(['input_documents', 'output_text'])

Output: 
1. Summarize the main ideas in 3 bullet points of the summary of the article. The article discusses the main ideas related to the main topic, which is a study that explores how traditional methods can clean up polluted soils and how oyster mushrooms can be used for this purpose. 2. What is the focus of the study on traditional methods?
The study is focused on traditional methods to clean up soil and address contaminants in the local environment, specifically examining how mushrooms can be used as a cleaning agent.
3. What are some examples of traditional methods that have been tested for their potential to remove pollutants from soils?
Some examples of traditional methods that have been tested for their potential to remove pollutants from soils include mycoremediation, which involves using mushrooms to break down chemicals and heavy metals. Another example is the use of bioaugmentation, which is a technique in which b

### [Chat Prompts](https://docs.llamaindex.ai/en/stable/examples/customization/prompts/chat_prompts/)

In [9]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.llm import LLMChain
from langchain_core.prompts import ChatPromptTemplate

In [10]:
# Define prompt
prompt = ChatPromptTemplate.from_messages(
    [("system", "Summarize the highlights of the following in 3 bullet points:\\n\\n{context}")]
)

In [11]:
# Instantiate chain
chain = create_stuff_documents_chain(llm, prompt)

In [13]:
# Invoke chain
result = chain.invoke({"context": pdf_data_RS})
print(result)



For more information about Tewa Women United, its work with Los Alamos and the community garden sites in Española, see tewawomen.org.
</s>


In [14]:
for token in chain.stream({"context": pdf_data_RS[1:2]}):
    print(token, end="|")

 “||we|| are|| not|| dis||connected|| from|| the|| people|| in|| our|| community|| and|| the|| water|| that|| supports|| our|| food|| systems||.”||T||s||os||ie||-||Pe||ña||,|| who|| is|| also|| a|| board|| member|| at|| T||ew||a|| Women|| United||,|| said|| that|| she|| bel||ieves|| that|| the|| land||mark|| resolution|| will|| bring|| attention|| to|| issues|| related|| to|| poll||ution|| on|| the|| N||ava||jo|| Nation||.|| She|| added|| that|| her|| organization|| has|| been|| working|| for|| years|| to|| establish|| water|| and|| air|| quality|| standards|| within|| trib||al|| communities||.|| This|| year||,|| Ts||os||ie||-||Pe||ña|| said||,|| “||we|| have|| a|| new|| opportunity||”|| to|| work|| with|| the|| state|| and|| federal|| govern||ments|| to|| improve|| those|| standards||.||</s>||||